In [ ]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(5) 
    return soup

In [ ]:
def get_data_ezinearticles_web(soup):
    
    articles=soup.find('div',attrs={'class':'category-list'}).find_all('div',attrs={'class':'article'})
    
    # Declaration
    articles_topic = []
    articles_content = []

    for article in articles:
        # Get article's topic
        topic = article.find("a").get_text()
        articles_topic.append(topic)
        
        # Get article's content - blocking us with lots of requests so we take the article's summary
        articles_content.append(article.find('div',attrs={'class':'article-summary'}).get_text()) 
   
    return articles_topic, articles_content

In [ ]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'ezinearticles Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["Business","Arts-and-Entertainment", "Computers-and-Technology", "Travel-and-Leisure", "Finance",
             "Health-and-Fitness", "Investing", "News-and-Society", "Communications", "Food-and-Drink",
              "Home-and-Family", "Pets"]

print("Start scraping from ezinearticles!")
for category in categories:
    driver = webdriver.Chrome()
    driver.get(f"https://ezinearticles.com/?cat={category}")
    driver.maximize_window()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    topic, content = get_data_ezinearticles_web(soup)
    insert_data_to_csv(topic, category, content)
    next_button = driver.find_element(by=By.LINK_TEXT, value=('Next »')).click() 
    for i in range (1, 29):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        topic, content = get_data_ezinearticles_web(soup)
        insert_data_to_csv(topic, category, content)
        next_button = driver.find_element(by=By.LINK_TEXT, value=('Next »')).click() 
driver.close()

print("Finish scraping from ezinearticles.com!")